In [ ]:
# Toy event loop
import queue as queue

from functools import partial

count = 0
eventloop = None

class EventLoop(queue.Queue):
    def start(self):
        while not self.empty():
            func = self.get()
            func()

def do_hello():
    global eventloop
    print("Hello")
    eventloop.put(do_world)

def do_world():
    global eventloop
    print("world")
    global count
    if count != 10:
        eventloop.put(do_hello)
        count += 1

eventloop = EventLoop()
eventloop.put(do_hello)
eventloop.start()

In [ ]:
import queue as queue
from functools import partial

eventloop = None
class EventLoop(queue.Queue):
    def start(self):
        while not self.empty():
            func = self.get()
            func()


def save_value(value, callback):
    print("Saving {} to database".format(value))
    save_result_to_db(value, callback)

def print_response(db_response):
    print("Response from database: {}".format(db_response))

def save_result_to_db(result, callback):
    callback(result)

eventloop = EventLoop()
eventloop.put(
    partial(save_value, "Hello World", print_response)
)
eventloop.start()

In [ ]:
import asyncio

eventloop = None
class EventLoop(queue.Queue):
    def start(self):
        while not self.empty():
            func = self.get()
            func()

def save_result_to_db(result, callback):
    return callback(result)
    
@asyncio.coroutine
def save_value(value, callback):
    print("Saving {} to database".format(value))
    db_response = yield save_result_to_db(value, callback)

def print_response(db_response):
    print("Response from database: {}".format(db_response))
    
eventloop = EventLoop()
eventloop.put(
    partial(save_value, "Hello World", print_response)
)
eventloop.start()

In [ ]:
import requests
import string
import random

def generate_urls(base_url, num_urls):
    """
    We add random characters to the end of the URL to break any caching
    mechanisms in the requests library or the server
    """
    for i in range(num_urls):
        yield base_url + "".join(random.sample(string.ascii_lowercase, 10))

def run_experiment(base_url, num_iter=500):
    response_size = 0
    for url in generate_urls(base_url, num_iter):
        response = requests.get(url)
        response_size += len(response.text)
    return response_size

import time
delay = 100
num_iter = 500
base_url = "http://127.0.0.1:8080/add?name=serial&delay={}&".format(delay)

start = time.time()
result = run_experiment(base_url, num_iter)
end = time.time()
print("Result: {}, Time: {}".format(result, end - start))

In [ ]:
from gevent import monkey
monkey.patch_socket()

import gevent
from gevent.lock import Semaphore
from urllib.request import urlopen
from contextlib import closing
import string
import random


def generate_urls(base_url, num_urls):
    for i in range(num_urls):
        yield base_url + "".join(random.sample(string.ascii_lowercase, 10))


def download(url, semaphore):
    with semaphore, closing(urlopen(url)) as data:
        return data.read()


def chunked_requests(urls, chunk_size=100):
    semaphore = Semaphore(chunk_size)
    requests = [gevent.spawn(download, u, semaphore) for u in urls]
    for response in gevent.iwait(requests):
        yield response


def run_experiment(base_url, num_iter=500):
    urls = generate_urls(base_url, num_iter)
    response_futures = chunked_requests(urls, 100)
    response_size = sum(len(r.value) for r in response_futures)
    return response_size


import time
delay = 100
num_iter = 500
base_url = "http://127.0.0.1:8080/add?name=gevent&delay={}&".format(delay)

start = time.time()
result = run_experiment(base_url, num_iter)
end = time.time()
print("Result: {}, Time: {}".format(result, end - start))

In [ ]:
import grequests
import string
import random


def generate_urls(base_url, num_urls):
    for i in range(num_urls):
        yield base_url + "".join(random.sample(string.ascii_lowercase, 10))


def run_experiment(base_url, num_iter=500):
    urls = generate_urls(base_url, num_iter)
    requests = (grequests.get(u) for u in urls)
    response_futures = grequests.imap(requests, size=100)
    response_size = sum(len(r.text) for r in response_futures)
    return response_size

import time
delay = 100
num_iter = 500

start = time.time()
result = run_experiment(
    "http://127.0.0.1:8080/add?name=grequests&delay={}&".format(delay),
    num_iter)
end = time.time()
print("Result: {}, Time: {}".format(result, end - start))

In [1]:
import asyncio
import aiohttp
import random
import string


def generate_urls(base_url, num_urls):
    for i in range(num_urls):
        yield base_url + "".join(random.sample(string.ascii_lowercase, 10))


def chunked_http_client(num_chunks):
    semaphore = asyncio.Semaphore(num_chunks)

    @asyncio.coroutine
    def http_get(url):
        nonlocal semaphore
        with (yield from semaphore):
            response = yield from aiohttp.request('GET', url)
            body = yield from response.content.read()
            yield from response.wait_for_close()
        return body
    return http_get


def run_experiment(base_url, num_iter=500):
    urls = generate_urls(base_url, num_iter)
    http_client = chunked_http_client(100)
    tasks = [http_client(url) for url in urls]
    responses_sum = 0
    for future in asyncio.as_completed(tasks):
        data = yield from future
        responses_sum += len(data)
    return responses_sum


import time
loop = asyncio.get_event_loop()
delay = 100
num_iter = 500

start = time.time()
result = loop.run_until_complete(
    run_experiment(
        "http://127.0.0.1:8080/add?name=asyncio&delay={}&".format(delay),
        num_iter))
end = time.time()
print("Result: {}, Time: {}".format(result, end - start))

Result: 14000, Time: 4.626115322113037


In [ ]:
from tornado import ioloop
from tornado.httpclient import AsyncHTTPClient
from tornado import gen

from functools import partial
import string
import random

AsyncHTTPClient.configure(
    "tornado.curl_httpclient.CurlAsyncHTTPClient", max_clients=100)


def generate_urls(base_url, num_urls):
    for i in xrange(num_urls):
        yield base_url + "".join(random.sample(string.ascii_lowercase, 10))


@gen.coroutine
def run_experiment(base_url, num_iter=500):
    http_client = AsyncHTTPClient()
    urls = generate_urls(base_url, num_iter)
    responses = yield [http_client.fetch(url) for url in urls]
    response_sum = sum(len(r.body) for r in responses)
    raise gen.Return(value=response_sum)


import time
delay = 100
num_iter = 500
_ioloop = ioloop.IOLoop.instance()
run_func = partial(
    run_experiment,
    "http://127.0.0.1:8080/add?name=tornado&delay={}&".format(delay),
    num_iter)

start = time.time()
result = _ioloop.run_sync(run_func)
end = time.time()
print("Result: {}, Time: {}".format(result, end - start))